In [1]:
from flask import Flask, request
from flask_sqlalchemy import SQLAlchemy
from sportsreference.nba.roster import Roster, Player
from sportsreference.nba.player import AbstractPlayer
from sportsreference.nba.teams import Teams
import pandas as pd

In [2]:
APP = Flask(__name__)
APP.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///nbadb.sqlite3'
APP.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
DB = SQLAlchemy(APP)
DB.init_app(APP)
# engine = create_engine('sqlite:///current.sqlite3')
# meta = MetaData(engine)

In [3]:
class NBAPlayer(DB.Model):
    __tablename__ = "nba_players"
#     __table_args__ = {"schema": "nbashcema"}
    ref_id = DB.Column(DB.String(10), primary_key=True, unique=True, nullable=False)
    name = DB.Column(DB.String(30), nullable=False)
    Team = DB.Column(DB.String(4), DB.ForeignKey("nba_teams.abbr"), nullable=False)
    FG_Pct = DB.Column(DB.Float, nullable=True)
    TP_Pct = DB.Column(DB.Float, nullable=True)
    FT_Pct = DB.Column(DB.Float, nullable=True)
    Min_per_game = DB.Column(DB.Float, nullable=False)
    Pts_per_game = DB.Column(DB.Float, nullable=False)
    TRB_per_game = DB.Column(DB.Float, nullable=False)
    Asts_per_game = DB.Column(DB.Float, nullable=False)
    WS_per_game = DB.Column(DB.Float, nullable=True)
    BPM = DB.Column(DB.Float, nullable=True)
    VORP = DB.Column(DB.Float, nullable=True)
    
    organization = DB.relationship("NBATeam")
    
    def __repr__(self):
        return '<NBAPlayer %r>' % self.username

In [4]:
class NBATeam(DB.Model):
    __tablename__ = "nba_teams"
#     __table_args__ = {"schema": "nbaschema"}
    abbr = DB.Column(DB.String(4), primary_key=True, unique=True, nullable=False)
    name = DB.Column(DB.String(30), unique=True, nullable=False)
    
    def __repr__(self):
        return '<NBATeam %r>' % self.name

In [5]:
import itertools

teams = Teams()
teamabbs = []
squadIDs=[]

for team in teams:
    teamabbs.append(team.abbreviation)

for abb in teamabbs:
    squad = Roster(abb, slim=True)
    squaddict = squad.players
    squadIDs.append(list(squaddict.keys()))

mergedIDs = list(itertools.chain.from_iterable(squadIDs))

In [6]:
teamabbs

['TOR',
 'MIN',
 'BRK',
 'CHO',
 'CHI',
 'PHO',
 'NOP',
 'SAS',
 'MIA',
 'DET',
 'LAC',
 'NYK',
 'IND',
 'DEN',
 'DAL',
 'PHI',
 'LAL',
 'MEM',
 'UTA',
 'POR',
 'WAS',
 'OKC',
 'SAC',
 'ORL',
 'BOS',
 'CLE',
 'MIL',
 'HOU',
 'GSW',
 'ATL']

In [7]:
mergedIDs[:25]

['gasolma01',
 'powelno01',
 'siakapa01',
 'anunoog01',
 'lowryky01',
 'ibakase01',
 'vanvlfr01',
 'bouchch01',
 'millema01',
 'hernade01',
 'mccawpa01',
 'johnsst04',
 'holliro01',
 'brissos01',
 'thomama02',
 'daviste02',
 'pondssh01',
 'dienggo01',
 'teaguje01',
 'wiggian01',
 'covinro01',
 'okogijo01',
 'bateske01',
 'napiesh01',
 'grahatr01']

In [9]:
def add_teams(teams):
    for team in teams:
        nbateam = NBATeam(abbr=team.abbreviation,
                          name=team.name)
        DB.session.add(nbateam)
    DB.session.commit()


In [10]:
def add_players(mergedIDs):
    for ref in mergedIDs:
#         print(ref)
        player = Player(ref)
        if hasattr(player, 'points') and player.games_played is not None:
#             print(ref, 'Has Atts. & games')
            x1 = player.field_goal_percentage
            x2 = player.three_point_percentage
            x3 = player.free_throw_percentage
            x4 = player.minutes_played/player.games_played
            x5 = player.points/player.games_played
            x6 = player.total_rebounds/player.games_played
            x7 = player.assists/player.games_played
            x8 = player.win_shares
            x9 = player.box_plus_minus
            x10 = player.value_over_replacement_player

            nbaplayer = NBAPlayer(ref_id=player.player_id,
                                  name=player.name,
                                  Team=player('2019-20').team_abbreviation,
                                  FG_Pct=x1,
                                  TP_Pct=x2,
                                  FT_Pct=x3,
                                  Min_per_game=x4,
                                  Pts_per_game=x5,
                                  TRB_per_game=x6,
                                  Asts_per_game=x7,
                                  WS_per_game=x8,
                                  BPM=x9,
                                  VORP=x10
                                  )
            DB.session.add(nbaplayer)
            DB.session.commit()

In [11]:
DB.create_all()

add_teams(teams)

In [12]:
add_players(mergedIDs)

In [ ]:
############################################
############################################

In [ ]:
# Everything below this line was test code

In [13]:
crosscols = ['field_goal_percentage', 'three_point_percentage', 'minutes_played', 'points', 'total_rebounds', 'assists', 'games_played', 'win_shares' ,'box_plus_minus', 'value_over_replacement_player', 'name']

list(Player('hardeja01').dataframe)

['and_ones',
 'assist_percentage',
 'assists',
 'block_percentage',
 'blocking_fouls',
 'blocks',
 'box_plus_minus',
 'center_percentage',
 'defensive_box_plus_minus',
 'defensive_rebound_percentage',
 'defensive_rebounds',
 'defensive_win_shares',
 'dunks',
 'effective_field_goal_percentage',
 'field_goal_attempts',
 'field_goal_perc_sixteen_foot_plus_two_pointers',
 'field_goal_perc_ten_to_sixteen_feet',
 'field_goal_perc_three_to_ten_feet',
 'field_goal_perc_zero_to_three_feet',
 'field_goal_percentage',
 'field_goals',
 'free_throw_attempt_rate',
 'free_throw_attempts',
 'free_throw_percentage',
 'free_throws',
 'games_played',
 'games_started',
 'half_court_heaves',
 'half_court_heaves_made',
 'height',
 'lost_ball_turnovers',
 'minutes_played',
 'nationality',
 'net_plus_minus',
 'offensive_box_plus_minus',
 'offensive_fouls',
 'offensive_rebound_percentage',
 'offensive_rebounds',
 'offensive_win_shares',
 'on_court_plus_minus',
 'other_turnovers',
 'passing_turnovers',
 'percen

In [22]:
DB.session.rollback()
DB.drop_all()
DB.metadata.clear()

/home/cicbeast/anaconda3/lib/python3.6/site-packages/sqlalchemy/orm/session.py:521: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [ ]:
minn = squadIDs[1]

In [12]:
minn

['dienggo01',
 'teaguje01',
 'wiggian01',
 'covinro01',
 'okogijo01',
 'bateske01',
 'napiesh01',
 'grahatr01',
 'townska01',
 'culveja01',
 'laymaja01',
 'vonleno01',
 'mclaujo01',
 'nowelja01',
 'martike03',
 'reidna01',
 'belljo01']

In [22]:
retrocks = ['mingya01', 'olajuha01']

In [11]:
# add_players(['mingya01'])

In [ ]:
DB.session.rollback

In [11]:
columns = ['player_id', 'field_goal_percentage', 'three_point_percentage', 'minutes_played', 'points', 'total_rebounds', 'assists', 'games_played', 'win_shares' ,'box_plus_minus', 'value_over_replacement_player']

blip = Player('hardeja01').dataframe[columns].tail(1)
blip = blip.append(Player('walljo01').dataframe[columns].tail(1), )

blip

,player_id,field_goal_percentage,three_point_percentage,minutes_played,points,total_rebounds,assists,games_played,win_shares,box_plus_minus,value_over_replacement_player
Career,hardeja01,0.443,0.365,26106,18627,4000,4743,765,121.9,7.1,59.9
Career,walljo01,0.433,0.324,20545,10879,2483,5282,573,44.3,2.3,22.1


In [29]:
Player('hardeja01').total_rebounds

4000

In [18]:
import sqlite3

conn = sqlite3.connect('nbadb.sqlite3')
cursor = conn.cursor()

nbads = pd.read_sql("select * from nba_players;", conn)

nbads.head()

,ref_id,name,Team,FG_Pct,TP_Pct,FT_Pct,Min_per_game,Pts_per_game,TRB_per_game,Asts_per_game,WS_per_game,BPM,VORP
0,gasolma01,Marc Gasol,TOR,0.483,0.349,0.777,33.413317,14.983668,7.685930,3.443467,79.5,3.4,36.3
1,powelno01,Norman Powell,TOR,0.441,0.343,0.807,16.835938,7.089844,2.132812,1.214844,7.3,-1.4,0.6
2,siakapa01,Pascal Siakam,TOR,0.529,0.310,0.748,23.612903,10.165899,5.142857,1.976959,15.7,2.3,5.5
3,anunoog01,OG Anunoby,TOR,0.461,0.350,0.600,20.204225,6.464789,2.732394,0.718310,5.0,-0.4,1.2
4,lowryky01,Kyle Lowry,TOR,0.423,0.367,0.805,31.302217,14.442240,4.269545,6.102684,86.8,3.8,39.3


In [14]:
import csv
import sqlite3
from glob import glob
from os.path import expanduser

conn = sqlite3.connect(glob(expanduser('nbadb.sqlite3'))[0])

cursor = conn.cursor()

cursor.execute("select * from nba_players;")

with open("nbads", "w", newline='') as csv_file:
    

TypeError: function takes at least 1 argument (0 given)

In [24]:
feats = ['player_id', 'field_goal_percentage', 'three_point_percentage']
nbapd = Player('walljo01').dataframe[feats].tail(1)
for squid in squadIDs:
    player = Player(squid)
    if squid == 'walljo01': 
        continue
    else:
        nbapd = nbapd.append(player.dataframe[feats].tail(1))
        
nbapd.sample(4)


KeyError: "None of [Index(['player_id', 'field_goal_percentage', 'three_point_percentage'], dtype='object')] are in the [columns]"

In [ ]:
# table1 = Table('nba_teams', meta,
#                DB.Column('abbr', DB.String(4), primary_key=True, unique=True, nullable=False),
#                DB.Column('name', DB.String(30), unique=True, nullable=False))

# table2 = Table('nba_players', meta,
#                DB.Column('ref_id', DB.String(10), primary_key=True, unique=True, nullable=False),
#                DB.Column('name', DB.String(30), nullable=False),
#                DB.Column('Team', DB.String(4), DB.ForeignKey("nbadb.nba_teams.abbr"), nullable=False),
#                DB.Column('FG_Pct', DB.Float, nullable=False), 
#                DB.Column('TP_Pct',DB.Float, nullable=False),
#                DB.Column('FT_Pct', DB.Float, nullable=False),
#                DB.Column('Min_per_game', DB.Float, nullable=False),
#                DB.Column('Pts_per_game', DB.Float, nullable=False),
#                DB.Column('TRB_per_game', DB.Float, nullable=False),
#                DB.Column('Asts_per_game', DB.Float, nullable=False),
#                DB.Column('WS_per_game', DB.Float, nullable=False),
#                DB.Column('BPM', DB.Float, nullable=False),
#                DB.Column('VORP', DB.Float, nullable=False))

# meta.crteate_all()